In [18]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds
import pathlib
import matplotlib.pyplot as plt
from tensorflow import keras
from keras import layers
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

dataset_url = "Datenbank/flower_photos"
data_dir = pathlib.Path(dataset_url)
image_count = len(list(data_dir.glob('*/*.jpg')))

## Create Image Batches

# Settings
test_split_size = 0.1
batch_size = 128
rescale_factor = 1/255
image_shape = (240, 320, 3)

image_gen = ImageDataGenerator(horizontal_flip=True,
                                    fill_mode='nearest',
                                    rescale=rescale_factor,
                                    validation_split=test_split_size)

train_dataset = image_gen.flow_from_directory(dataset_url,
                                             target_size = image_shape[:2],
                                             batch_size=batch_size,
                                             class_mode='categorical',
                                             subset='training',
                                             color_mode='rgb',
                                             seed = 300,
                                             shuffle=True)


validation_dataset = image_gen.flow_from_directory(dataset_url,
                                                  target_size = image_shape[:2],
                                                  batch_size=batch_size,
                                                  class_mode='categorical',
                                                  subset='validation',
                                                  color_mode='rgb',
                                                  seed = 300,
                                                  shuffle=False)

train_dataset.class_indices

Found 3306 images belonging to 5 classes.
Found 365 images belonging to 5 classes.


{'daisy': 0, 'dandelion': 1, 'roses': 2, 'sunflowers': 3, 'tulips': 4}

In [19]:
## BUILDING A MODEL
early_stop = EarlyStopping(monitor='val_loss', patience=0)
# Defining Number Of Steps For Training & Validation Of Model
steps_per_epoch = train_dataset.samples // batch_size
validation_steps = validation_dataset.samples // batch_size

print('Steps Per Epoch : Training -> ' + str(steps_per_epoch))
print('Steps : Validation -> ' + str(validation_steps))

Steps Per Epoch : Training -> 25
Steps : Validation -> 2


In [21]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout

model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(4,4), input_shape = image_shape, activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=64, kernel_size=(4,4), activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=96, kernel_size=(4,4), activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(5, activation='softmax'))

model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 237, 317, 32)      1568      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 118, 158, 32)     0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 115, 155, 64)      32832     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 57, 77, 64)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 54, 74, 96)        98400     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 27, 37, 96)      

In [23]:
history = model.fit(train_dataset,epochs=3,steps_per_epoch = steps_per_epoch,
                    validation_data = validation_dataset, validation_steps=validation_steps,
                    verbose=True, callbacks=[early_stop])

Epoch 1/3
25/25 [==============================] - 138s 6s/step - loss: 1.4344 - accuracy: 0.3760 - val_loss: 1.4100 - val_accuracy: 0.3867
Epoch 2/3
25/25 [==============================] - 145s 6s/step - loss: 1.1572 - accuracy: 0.5205 - val_loss: 1.1720 - val_accuracy: 0.6250


In [24]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

In [26]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model.h5")

In [27]:
model_eval_metrics = model.evaluate_generator(validation_dataset, steps=validation_steps, verbose=1)
print('Model Accuracy : ' + str(round(model_eval_metrics[1] * 100, 2)))
print('Model Loss : ' + str(round(model_eval_metrics[0],2)))

/var/folders/kp/z9j20h853bq1d376dvh_pyvc0000gn/T/ipykernel_23526/2876603078.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  model_eval_metrics = model.evaluate_generator(validation_dataset, steps=validation_steps, verbose=1)


2/2 [==============================] - 3s 1s/step - loss: 1.1806 - accuracy: 0.6055
Model Accuracy : 60.55
Model Loss : 1.18
